## Using Presto to Generate Reports

To generate reports for those interested in application usage, we can use Presto via the PyHive connector to load our Hive tables into Pandas dataframes, transform the dataframes to answer our business questions, and then publish the reports as JSON files.

Before we start, let's define two business questions that we would like to answer:

1. What are all the counts per event type?
2. What are all the parameters that were given for the `user` parameter?

First, let's install the PyHive library.

In [1]:
!python3 -m pip install git+https://github.com/dropbox/PyHive.git > null

The directory '/w205/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/w205/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 8.1.1, however version 23.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Next, let's use PyHive to connect to Presto in code using the port that we exposed in our Docker Compose file. Once we connect, we can run a simple query to see all the tables that are created in Hive.

In [4]:
from pyhive import presto
import pandas as pd

presto_conn = presto.connect(
    host='presto',
    port=8080 # Exposed Presto port (see docker compose file)
)

pd.read_sql_query("SHOW TABLES", presto_conn)

,Table
0,all_events
1,event_parameters


Now let's run a query to get all of the data from the `event_parameters` table and load it into a Pandas dataframe.

In [5]:
# https://stackoverflow.com/questions/55988436/how-to-convert-a-presto-query-output-to-a-python-data-frame
event_parameters = pd.read_sql_query("SELECT * from event_parameters", presto_conn)
event_parameters.head()

,raw_event,timestamp,accept,host,user_agent,event_id,parameter_name,parameter_value
0,"{""event_id"": ""2e8d0e2c-033f-40f1-941e-96354476...",2023-08-18 08:20:56.096,None,None,None,2e8d0e2c-033f-40f1-941e-96354476e7d0,user,theresa
1,"{""event_id"": ""2e8d0e2c-033f-40f1-941e-96354476...",2023-08-18 08:20:56.097,None,None,None,2e8d0e2c-033f-40f1-941e-96354476e7d0,guild_name,Knights of the Scorpion
2,"{""event_id"": ""9fdabe7f-9ab7-447f-97d3-ae114c17...",2023-08-18 08:20:56.099,None,None,None,9fdabe7f-9ab7-447f-97d3-ae114c17028c,user,theresa
3,"{""event_id"": ""9fdabe7f-9ab7-447f-97d3-ae114c17...",2023-08-18 08:20:56.099,None,None,None,9fdabe7f-9ab7-447f-97d3-ae114c17028c,guild_name,Knights of the Scorpion
4,"{""event_id"": ""85ad5ad2-4ca0-455f-9d8d-5e303663...",2023-08-18 08:20:56.102,None,None,None,85ad5ad2-4ca0-455f-9d8d-5e3036633347,user,theresa


Let's do the same thing for the data in the `all_events` table.

In [6]:
all_events = pd.read_sql_query("SELECT * from all_events", presto_conn)
all_events.head()

,raw_event,timestamp,accept,host,user_agent,event_id,event_type
0,"{""event_id"": ""b2f47cf2-9f58-4b98-baf5-0690bbeb...",2023-08-18 08:21:10.082,*/*,user2.att.com,ApacheBench/2.3,b2f47cf2-9f58-4b98-baf5-0690bbeb2cb5,get_credit
1,"{""event_id"": ""1c91e161-20c5-462a-855e-7b7c25e8...",2023-08-18 08:21:10.084,*/*,user2.att.com,ApacheBench/2.3,1c91e161-20c5-462a-855e-7b7c25e841f4,get_credit
2,"{""event_id"": ""49b79627-7671-4d7f-bd9d-9a927fc3...",2023-08-18 08:21:10.086,*/*,user2.att.com,ApacheBench/2.3,49b79627-7671-4d7f-bd9d-9a927fc36c80,get_credit
3,"{""event_id"": ""b3e6df78-981a-4098-811b-f0251c2b...",2023-08-18 08:21:10.088,*/*,user2.att.com,ApacheBench/2.3,b3e6df78-981a-4098-811b-f0251c2b2f4b,get_credit
4,"{""event_id"": ""bf3be374-c95a-4fed-826d-ad1b343c...",2023-08-18 08:21:10.09,*/*,user2.att.com,ApacheBench/2.3,bf3be374-c95a-4fed-826d-ad1b343c9c67,get_credit


Now let's try to answer business question #1 - What are all the counts per event type? We can do this using a simple groupby statement on our `all_events` dataframe and then writing the output to a JSON file titled `event_type_count.json`.

In [7]:
event_type_count = all_events.groupby('event_type').size()
event_type_count.to_json("event_type_count.json", orient='columns')

Note that `event_type_count.json` should exist in your directory after running the code above.


In [8]:
all_events = pd.read_sql_query("select event_type, count(event_type) as event_count from all_events group by event_type", presto_conn)
all_events.head()

,event_type,event_count
0,get_credit,1062
1,leave_guild,1267
2,join_guild,1038
3,purchase_sword,1135




Now let's answer business question #2 - What are all the parameters that were given for the `user` parameter? We can do this by running a slightly more complex query on our dataframe and again writing the output to a JSON file.

In [9]:
# Question: What are all the parameters that were given for the `user` parameter?
user_parameter_count = event_parameters.where(event_parameters['parameter_name'] == 'user').groupby('parameter_value').size()
user_parameter_count.to_json("user_parameter_count.json", orient='columns')

events_by_user = pd.read_sql_query(" select     un.parameter_value as user,     et.event_type as event,    count(un.parameter_value) as user_event_count from     all_events et join      event_parameters un on      et.event_id = un.event_id and     un.parameter_name = 'user' where      un.parameter_name = 'user'\
and     et.event_id = un.event_id group by     un.parameter_value     ,et.event_type order by     et.event_type,     un.parameter_value ", presto_conn)

events_by_user.head(50)


,user,event,user_event_count
0,aastha,get_credit,232
1,ben,get_credit,245
2,don,get_credit,229
3,lise,get_credit,216
4,theresa,get_credit,341
5,aastha,join_guild,149
6,ben,join_guild,316
7,don,join_guild,239
8,lise,join_guild,243
9,theresa,join_guild,181


Now that we have answered the business questions and generated JSON reports with the answers, we can close our Presto connection.

In [24]:
presto_conn.close()